In [46]:
import re
import os
logpath = "./log"
tuningpath = './tuning'
filenames = os.listdir(logpath)

for filen in filenames:
    read_f = os.path.join(logpath,filen)
    write_f = os.path.join(tuningpath,f"{filen}.csv")
    result = []
    with open(read_f, 'r') as f:
        for l in f.readlines():
            newl = l.strip('\n')
            # print(newl)
            if len(newl.strip(" ")) > 0:
                match1 = re.search(r"\[Epoch [0-9]{1,3} valid\]",newl)
                match2 = re.search(r"\[Epoch [0-9]{1,3} test\]",newl)
                if (match1 and "PPL: Tail" not in newl) \
                or newl.split(" ")[1] == "topic_recon:" \
                or (match2 and "PPL: Tail" not in newl):
                    result.append(newl)

    # len(result)
    # Epoch,split,PPL,topic_recon,topic_coherence_drug-drug,topic_coherence_disease-disease,topic_coherence_drug-disease
    cols = "Epoch,split,PPL,topic_recon,topic_coherence_drug-drug,topic_coherence_disease-disease,topic_coherence_drug-disease\n"
    data = []
    for i in range(0,len(result)-1,2):
        line1 = result[i]
        line2 = result[i+1]
        
        # extracting info from line1: epoch num, split, and PPL
        match1 = re.search(r"Epoch [0-9]{1,3} valid",line1)
        match2 = re.search(r"Epoch [0-9]{1,3} test",line1)
        if match1:
            _,epoch,split = match1.group().split(" ")
        elif match2:
            _,epoch,split = match2.group().split(" ")
        ppl = re.search(r"PPL:.*",line1).group().split(" ")[1]
        
        # extracting info from line2:
        tokens = line2.split(" ")
        topic_re = tokens[2]
        tc_drdr = tokens[6]
        tc_didi = tokens[10]
        tc_drdi = tokens[-1]
        towrite = f"{epoch},{split},{ppl},{topic_re},{tc_drdr},{tc_didi},{tc_drdi}\n"
        data.append(towrite)

    with open(write_f,"w") as f:
        f.write(cols)
        f.writelines(data)